In [1]:
# import libraries and dependencies
import os
import requests
import mysql.connector
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import smtplib

from dotenv import load_dotenv
from mysql.connector import Error
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
# load environment variables from .env file
load_dotenv()

True

In [3]:
# api endpoints and keys
TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')

TICKETMASTER_Events_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/events.json"
# Note: Austin’s weather API endpoint will be constructed via parameters below.

In [4]:
# MySQL connection parameters
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [5]:
# function to handle the database connection
def get_db_connection():
    """Establish and return a MySQL database connection."""
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DATABASE
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Database connection error: {e}")
    return None

In [6]:
# Function to extract full datetime from event 'dates' field
def extract_event_datetime(dates):
    """
    Extract a full datetime string from the Ticketmaster API's dates field.
    Expects 'dates' to be a dict with a 'start' key that contains 'localDate'
    and optionally 'localTime'. If 'localTime' is missing, defaults to "12:00:00".
    Returns a string formatted as 'YYYY-MM-DD HH:MM:SS' or None if not available.
    """
    if isinstance(dates, dict) and 'start' in dates:
        start_info = dates['start']
        local_date = start_info.get('localDate')
        # Attempt to get the localTime; if missing, default to noon
        local_time = start_info.get('localTime', "12:00:00")
        if local_date:
            try:
                dt = datetime.strptime(f"{local_date} {local_time}", "%Y-%m-%d %H:%M:%S")
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            except Exception as e:
                print(f"Error parsing datetime: {e}")
                return None
    return None

In [7]:
# cynthias events and venues code in a function
def get_events_and_venues_data():
    """
    Pull Ticketmaster events for San Diego for the next 7 days,
    then parse and return two DataFrames:
      - events_df: event_id, event_name, start_date, venue_id, event_category
      - venues_df: venue_id, venue_name, venue_city, venue_state, venue_country, venue_location
    """
    # define date range: today to 6 days later (7 days total)
    today = datetime.today()
    six_days_later = today + timedelta(days=5)
    start_date = today.strftime('%Y-%m-%dT00:00:00Z')
    end_date = six_days_later.strftime('%Y-%m-%dT23:59:59Z')
    
    params = {
        "apikey": TICKETMASTER_API_KEY,
        "city": "San Diego",
        "size": 200,
        "startDateTime": start_date,
        "endDateTime": end_date
    }
    
    print("Requesting Ticketmaster events...")
    response = requests.get(TICKETMASTER_Events_API_Endpoint, params=params)
    if response.status_code != 200:
        raise Exception(f"Ticketmaster API Error: {response.status_code}, {response.text}")
    
    data = response.json()
    events = data.get("_embedded", {}).get("events", [])
    if not events:
        print("No events found.")
        return pd.DataFrame(), pd.DataFrame()
    
    # convert list of events to dataframe
    df = pd.DataFrame(events)
    
    # extract event details
    df['event_id'] = df['id']
    df['event_name'] = df['name']
    # Use 'dates.start.localDate' as our start_date (could be refined to include time if available)
    df['start_date'] = df['dates'].apply(extract_event_datetime)
    df['event_category'] = df['classifications'].apply(lambda x: x[0]['segment']['name'] if isinstance(x, list) and len(x) > 0 and 'segment' in x[0] else None)
    
    # extract Venue details
    df['venue_id'] = df['_embedded'].apply(lambda x: x['venues'][0]['id'] if isinstance(x, dict) and 'venues' in x else None)
    df['venue_name'] = df['_embedded'].apply(lambda x: x['venues'][0]['name'] if isinstance(x, dict) and 'venues' in x else None)
    df['venue_city'] = df['_embedded'].apply(lambda x: x['venues'][0]['city']['name'] if isinstance(x, dict) and 'venues' in x else None)
    df['venue_state'] = df['_embedded'].apply(lambda x: x['venues'][0]['state']['stateCode'] if isinstance(x, dict) and 'venues' in x and 'state' in x['venues'][0] else None)
    df['venue_country'] = df['_embedded'].apply(lambda x: x['venues'][0]['country']['countryCode'] if isinstance(x, dict) and 'venues' in x else None)
    # combine latitude and longitude into one "venue_location" column (as requested)
    df['venue_location'] = df['_embedded'].apply(
        lambda x: f"{x['venues'][0]['location']['latitude']},{x['venues'][0]['location']['longitude']}" 
        if isinstance(x, dict) and 'venues' in x and 'location' in x['venues'][0] else None
    )
    
    # create dataframes for events and venues
    events_df = df[['event_id', 'event_name', 'start_date', 'venue_id', 'event_category']]
    venues_df = df[['venue_id', 'venue_name', 'venue_city', 'venue_state', 'venue_country', 'venue_location']].drop_duplicates()
    
    print(f"Pulled {len(events_df)} events from API.")
    return events_df, venues_df

In [8]:
# inserting cynthias events and venues data into the database
def insert_events_and_venues(events_df, venues_df, connection):
    """
    Insert venues and events data into MySQL.
    Before insertion, delete any past events.
    """
    try:
        cursor = connection.cursor()
        
        # delete past events (using 'start_date' column)
        delete_query = "DELETE FROM events WHERE start_date < CURDATE();"
        cursor.execute(delete_query)
        print(f"Deleted {cursor.rowcount} past events.")
        connection.commit()
        
        # insert/update venues
        insert_venue_query = """
        INSERT INTO venues (venue_id, name, city, state, country, location)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            name = VALUES(name),
            city = VALUES(city),
            state = VALUES(state),
            country = VALUES(country),
            location = VALUES(location);
        """
        for _, row in venues_df.iterrows():
            cursor.execute(insert_venue_query, tuple(row))
        connection.commit()
        print("Venues inserted/updated successfully.")
        
        # insert/update events
        # note: our events table expects columns: event_id, name, start_date, venue_id, and a category column.
        insert_event_query = """
        INSERT INTO events (event_id, name, start_date, venue_id, category)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            name = VALUES(name),
            start_date = VALUES(start_date),
            venue_id = VALUES(venue_id),
            category = VALUES(category);
        """
        for _, row in events_df.iterrows():
            cursor.execute(insert_event_query, tuple(row))
        connection.commit()
        print("Events inserted/updated successfully.")
        
        cursor.close()
    except Error as e:
        print(f"Error inserting events/venues: {e}")

In [9]:
# austins weather code in a function
def get_weather_forecast_data():
    """
    Pull hourly weather forecasts for San Diego for the next week.
    Process the data into a DataFrame that matches the new weather_forecasts schema.
    Schema includes:
        forecast_time (DATETIME),
        forecast_temperature (DECIMAL),
        forecast_temp_apparent (DECIMAL),
        forecast_humidity (DECIMAL),
        rain_intensity (DECIMAL),
        forecast_winds (DECIMAL),
        weather_code (INT),
        weather_icon (TEXT)
    """
    # define date range (today to 6 days later)
    today = datetime.today()
    next_7_days = today + timedelta(days=5)
    start_date = today.strftime('%Y-%m-%dT00:00:00Z')
    end_date = next_7_days.strftime('%Y-%m-%dT23:59:59Z')
    
    print("Requesting Tomorrow.io weather forecast data...")
    params = {
        "apikey": TOMORROW_API_KEY,
        "location": "32.7157,-117.1611",
        "startDateTime": start_date,
        "endDateTime": end_date
    }
    # construct endpoint url (Austin’s hard-coded an endpoint; here we use it)
    endpoint = "https://api.tomorrow.io/v4/weather/forecast"
    response = requests.get(endpoint, params=params)
    
    if response.status_code != 200:
        print(f"Weather API call failed: {response.status_code}, {response.text}")
        return pd.DataFrame()
    
    data = response.json()
    
    # parse the weather forecast data.
    # assume the JSON structure contains an hourly timeline under data['timelines']['hourly']
    weather_data = []
    # define mapping from weather code to icon filename (as provided)
    weather_code_icons = {
        1100: "mostly_clear_day.svg",
        1101: "partly_cloudy_day.svg",
        1102: "mostly_cloudy.svg",
        1000: "clear_day.svg",
        1001: "cloudy.svg",
        2100: "fog_light.svg",
        2000: "fog.svg",
        4000: "drizzle.svg",
        4200: "rain_light.svg",
        4001: "rain.svg",
        4201: "rain_heavy.svg",
        6000: "freezing_drizzle.svg",
        6200: "freezing_rain_light.svg",
        6001: "freezing_rain.svg",
        6201: "freezing_rain_heavy.svg",
        8000: "tstorm.svg",
    }
    base_url = "https://raw.githubusercontent.com/Tomorrow-IO-API/tomorrow-weather-codes/master/V1_icons/color/"
    
    # loop over hourly forecasts
    try:
        intervals = data['timelines']['hourly']
    except KeyError:
        print("Unexpected weather API response structure.")
        return pd.DataFrame()
    
    for interval in intervals:
        # each interval should have a 'time' and 'values'
        time_str = interval.get('time')
        # convert time string (assumed in ISO format) to MySQL DATETIME string format
        forecast_time = pd.to_datetime(time_str, utc=True).tz_convert("America/Los_Angeles").strftime('%Y-%m-%d %H:%M:%S')
        values = interval.get('values', {})
        forecast_temperature = values.get('temperature')
        forecast_temp_apparent = values.get('temperatureApparent')
        forecast_humidity = values.get('humidity')
        rain_intensity = values.get('rainIntensity')
        forecast_winds = values.get('windSpeed')
        weather_code = values.get('weatherCode')
        #get icon based on weather_code mapping
        icon_filename = weather_code_icons.get(weather_code, "default.png")
        weather_icon = f"{base_url}{icon_filename}"
        
        weather_data.append({
            'forecast_time': forecast_time,
            'forecast_temperature': forecast_temperature,
            'forecast_temp_apparent': forecast_temp_apparent,
            'forecast_humidity': forecast_humidity,
            'rain_intensity': rain_intensity,
            'forecast_winds': forecast_winds,
            'weather_code': weather_code,
            'weather_icon': weather_icon
        })
    
    weather_df = pd.DataFrame(weather_data)
    print(f"Retrieved {len(weather_df)} hourly weather records.")
    return weather_df

In [10]:
# create a function to delete all weather data from the weather_forecasts table
def delete_all_weather_data(cursor):
    """
    This function deletes the contents of the weather_forecasts table.
    This ensures that there is no duplicate weather forecasts that can cause confusion.
    Ensures only fresh forecast data is inserted and present each time the ELT pipeline runs.
    """
    delete_weather_query = "TRUNCATE TABLE weather_forecasts;"
    cursor.execute(delete_weather_query)

In [11]:
# inserting austins weather data into the database
def insert_weather_forecast_data(weather_df, connection):
    """Insert weather forecast data into the weather_forecasts table."""
    try:
        cursor = connection.cursor()
        insert_query = """
            INSERT INTO weather_forecasts (
                forecast_time, forecast_temperature, forecast_temp_apparent, 
                forecast_humidity, rain_intensity, forecast_winds, weather_code, weather_icon
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        for row in weather_df.itertuples(index=False):
            cursor.execute(insert_query, (
                row.forecast_time, row.forecast_temperature, row.forecast_temp_apparent,
                row.forecast_humidity, row.rain_intensity, row.forecast_winds, 
                row.weather_code, row.weather_icon
            ))
        connection.commit()
        print(f"Inserted {cursor.rowcount} weather forecast records.")
        cursor.close()
    except Error as e:
        print(f"Error during weather data insertion: {e}")

In [12]:
# determine if we want this, but this is the visualization breakout that was in cynthias code
def visualize_data(events_df):
    """
    (optional) Create sample visualizations for events.
    For example, plot number of events per venue and events by category.
    """
    # group by venue_name and count unique events
    event_counts = events_df.groupby('event_name').size().reset_index(name='count')
    event_counts = event_counts.sort_values(by='count', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.bar(event_counts['event_name'], event_counts['count'], color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Event')
    plt.ylabel('Count')
    plt.title('Events Count')
    plt.tight_layout()
    plt.show()

In [13]:
# create query in order to get the event, venue and forecast details
def get_event_venue_weather_details():
    """
    Retrieves event, venue, and weather forecast info for the events
    """
    conn = get_db_connection()
    query = """
    SELECT 
        e.name AS Event, 
        e.start_date AS DateTime,
        v.name AS Venue,
        (wf.forecast_temperature * 9 / 5 + 32) AS Temp_F,
        (wf.forecast_temp_apparent * 9 / 5 + 32) AS Feels_Like_F,
        wf.forecast_humidity AS Humidity,
        wf.rain_intensity AS Rain_mm,
        ROUND((wf.forecast_winds * 0.621371), 2) AS Winds_mph,
        wf.weather_icon
    FROM events AS e
    JOIN venues AS v on e.venue_id = v.venue_id
    LEFT JOIN weather_forecasts AS wf 
        ON wf.forecast_time BETWEEN e.start_date - INTERVAL 29 MINUTE 
                            AND e.start_date + INTERVAL 30 MINUTE
    WHERE e.start_date >= NOW() + INTERVAL -8 HOUR
    ORDER BY e.start_date;
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [14]:
# function that prepares an email message with the event, venue and weather details for the week
def send_notification(joined_df):
    """
    Sends email notification with the event and weather forecast details for the time of the event
    
    Inputs:
        joined_df: pandas df witih output of events and weather details
        
    Env Variables
        Credentials for email account and recipients
    """
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # email account credentials
    sender_email = os.getenv('SENDER_EMAIL')
    sender_password = os.getenv('SENDER_PASSWORD')
    recipient_emails = os.getenv('RECIPIENT_EMAILS')
    print(f"Sending email to: {recipient_emails}")
    # check for env credentials are valid and not empty
    if not sender_email or not sender_password or not recipient_emails:
        print("SMTP credentials or recipeint emails not provided or set in ENV variables.")
        return
    
    # email subject
    subject = "Upcoming Events and Weather Forecasts for San Diego"

    # convert DF to HTML table
    if joined_df.empty:
        body_html = "<p>No upcoming events found.</p>"
    else:
        # adds the part that goes to the github and grabs the picture.
        def embed_icon(url):
            if url:
                return f'<img src="{url}" width="40" height="40" alt="weather_icon"/>'
            else:
                return "No Icon Available"
        
        joined_df['weather_icon'] = joined_df['weather_icon'].apply(embed_icon)

        html_table = joined_df.to_html(index=False, border=1, escape=False)
        body_html = f"""
        <html>
            <head>
                <style>
                    /* Add basic styling for the table */
                    table {{
                        border-collapse: collapse;
                        width: 100%;
                    }}
                    th, td {{
                        border: 1px solid #dddddd;
                        text-align: left;
                        padding: 8px;
                    }}
                    th {{
                        background-color: #f2f2f2;
                    }}
                </style>
            </head>
            <body>
                <h2>Upcoming San Diego Events with Weather Forecasts</h2>
                {html_table}
            </body>
        </html>
        """
    
    # create the actual message headers subject, from, to
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = recipient_emails

    # attach the message body
    msg.attach(MIMEText(body_html, 'html'))

    # connect gmail server and send email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            # start TLS for security
            server.starttls()
            #login to email account
            server.login(sender_email, sender_password)
            #send email and split recipients by comma
            server.sendmail(sender_email, recipient_emails.split(','), msg.as_string())
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

In [15]:
def main():
    connection = get_db_connection()
    if not connection:
        print("Exiting due to database connection error.")
        return

    # Process events and venues
    try:
        events_df, venues_df = get_events_and_venues_data()
        if not events_df.empty and not venues_df.empty:
            insert_events_and_venues(events_df, venues_df, connection)
        else:
            print("No events/venues data to insert.")
    except Exception as e:
        print(f"Error during events/venues processing: {e}")
    
    # Process weather forecasts
    try:
        cursor = connection.cursor()
        delete_all_weather_data(cursor)
        connection.commit()
        weather_df = get_weather_forecast_data()
        if not weather_df.empty:
            insert_weather_forecast_data(weather_df, connection)
        else:
            print("No weather forecast data to insert.")
    except Exception as e:
        print(f"Error during weather forecast processing: {e}")
    
    connection.close()
    
    # Retrieve joined data and send notification email
    try:
        joined_df = get_event_venue_weather_details()
        send_notification(joined_df)
    except Exception as e:
        print(f"Error sending notification email: {e}")
    
    print("ETL pipeline executed successfully.")

if __name__ == "__main__":
    main()

Connected to MySQL database
Requesting Ticketmaster events...
Pulled 57 events from API.
Deleted 13 past events.
Venues inserted/updated successfully.
Events inserted/updated successfully.
Requesting Tomorrow.io weather forecast data...
Retrieved 120 hourly weather records.
Inserted 1 weather forecast records.
Connected to MySQL database


C:\Users\graha\AppData\Local\Temp\ipykernel_64432\3198284887.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Sending email to: grahamward92@gmail.com,grahamward@me.com
Email sent successfully.
ETL pipeline executed successfully.
